In [1]:
import numpy as np
import plotly.graph_objects as go
from scipy.sparse import csr_matrix

Task №15
\begin{equation*}
    \begin{cases}
        u_{xx}+u_{yy}=f(x,y), \quad x\in (0,1), \quad y\in (0,2)\\
        u(x,0)=u_{true}(x,0)\\
        u(x,2)=u_{true}(x,2)\\
        u(0,y)=u_{true}(0,y)\\
        u(1,y)=u_{true}(1,y)\\
    \end{cases}
\end{equation*}

\begin{equation*}
    u_{true}(x,y)=(3x+2y)\sin(5x+y)-3x^2-y^2
\end{equation*}

\begin{equation*}
    f(x,y)=\frac{\partial ^2u_{true}}{\partial x^2}+\frac{\partial ^2u_{true}}{\partial y^2}=-26(3x+2y)\sin(5x+y)+34\cos(5x+y)-8
\end{equation*}

Variables for the task №15

In [2]:
# Define x_min, x_max, y_min, y_max
x_min = 0
x_max = 1
y_min = 0
y_max = 2

# Define numbers of points
Nx = 100
Ny = 100

# Define u_true
def u_true(x, y):
    return (3*x+2*y)*np.sin(5*x+y)-3*x**2-y**2

# Define f
def f(x, y):
    return -26*(3*x+2*y)*np.sin(5*x+y)+34*np.cos(5*x+y)-8

# Define border_x_min
def border_x_min(y):
    return u_true(x_min, y)

# Define border_x_max
def border_x_max(y):
    return u_true(x_max, y)

# Define border_y_min
def border_y_min(x):
    return u_true(x, y_min)

# Define border_y_max
def border_y_max(x):
    return u_true(x, y_max)

Surface of true solution

In [3]:
x = np.linspace(x_min, x_max, Nx)
y = np.linspace(y_min, y_max, Ny)
X, Y = np.meshgrid(x, y)
Z = u_true(X, Y)
fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y)])
fig.show()

Minumum error method for solving linear systems

In [4]:
def min_error(A, b):
    x = np.zeros(len(b))
    A = csr_matrix(A)
    r = A.dot(x) - b
    while np.linalg.norm(r) > 10**(-5):
        r = A.dot(x) - b
        x -= r.dot(r) / r.dot(A.dot(r)) * r
    return x

Minimal residual method for solving linear systems

In [5]:
def min_res(A, b):
    x = np.zeros(len(b))
    A = csr_matrix(A)
    r = A.dot(x) - b
    while np.linalg.norm(r) > 10**(-5):
        r = A.dot(x) - b
        Ar = A.dot(r)
        x -= Ar.dot(r) / Ar.dot(Ar) * r
    return x

Function that returns A and b for the system of linear equations for Poisson equation

In [6]:
def GetPoissonSLE(x_min, x_max, y_min, y_max, border_x_min, border_x_max, border_y_min, border_y_max, f, Nx, Ny):
    hx = (x_max - x_min) / (Nx - 1)
    hy = (y_max - y_min) / (Ny - 1)
    N1 = Nx - 2
    N2 = Ny - 2
    A = np.zeros((N1 * N2, N1 * N2))
    b = np.zeros(N1 * N2)
    for i in range(N2):
        for j in range(N1):
            k = i * N1 + j
            A[k, k] = 2 / hx**2 + 2 / hy**2
            if j > 0:
                A[k, k - 1] = -1/hx**2
            else:
                b[k] += 1/hx**2 * border_x_min(y_min + (i+1) * hy)
            if j < N1 - 1:
                A[k, k + 1] = -1/hx**2
            else:
                b[k] += 1/hx**2 * border_x_max(y_min + (i+1) * hy)
            if i > 0:
                A[k, k - N1] = -1/hy**2
            else:
                b[k] += 1/hy**2 * border_y_min(x_min + (j+1) * hx)
            if i < N2 - 1:
                A[k, k + N1] = -1/hy**2
            else:
                b[k] += 1/hy**2 * border_y_max(x_min + (j+1) * hx)
            b[k] += -f(x_min + (j+1) * hx, y_min + (i+1) * hy)
    return A, b

Function of finding the solution of Poisson equation

In [7]:
def MakeFullSolution(x_min, x_max, y_min, y_max, border_x_min, border_x_max, border_y_min, border_y_max, f, Nx, Ny, solveSLE):
    hx = (x_max - x_min) / (Nx - 1)
    hy = (y_max - y_min) / (Ny - 1)
    N1 = Nx - 2
    N2 = Ny - 2
    A, b = GetPoissonSLE(x_min, x_max, y_min, y_max, border_x_min, border_x_max, border_y_min, border_y_max, f, Nx, Ny)
    u = solveSLE(A, b)
    U = np.zeros((Nx, Ny))
    for i in range(N2):
        for j in range(N1):
            U[j+1][i+1] = u[i * N1 + j]
    for i in range(Nx):
        U[i][0] = border_y_min(x_min + i * hx)
        U[i][Ny-1] = border_y_max(x_min + i * hx)
    for i in range(Ny):
        U[0][i] = border_x_min(y_min + i * hy)
        U[Nx-1][i] = border_x_max(y_min + i * hy)
    return U.T

Surface of numerical solution

In [8]:
U = MakeFullSolution(x_min, x_max, y_min, y_max, border_x_min, border_x_max, border_y_min, border_y_max, f, Nx, Ny, min_res)
x = np.linspace(x_min, x_max, Nx)
y = np.linspace(y_min, y_max, Ny)
X, Y = np.meshgrid(x, y)
fig = go.Figure(data=[go.Surface(z=U, x=X, y=Y)])
fig.show()

Error surface and maximum error

In [9]:
error = np.max(abs(U - u_true(X, Y)))
print('Max error = ', error)
fig = go.Figure(data=[go.Surface(z=U - u_true(X, Y), x=X, y=Y)])
fig.show()

Max error =  0.0010067289392310386
